# notebook for decoding iss data

## import packages

In [1]:
import align_2d_tiles
import stitch_2d_mist
import tile_stitched_images
#import test_analysis

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import glob
import tifffile
from xml.dom import minidom
import pandas as pd
import numpy as np
import os

## find the samples/sections to decode

In [3]:
input_dir = 'D:\Processing/'
sections = listdir(input_dir)
sections

['588B_LNG2',
 '689C_LNG1',
 '689C_LNG2',
 '689C_LNG3',
 '689C_LNG5',
 'Discovair_codebook.csv']

### in this section we will specify which sections we **DO NOT** want to process

In [4]:
sections_to_remove = ['Cycle_1', 'Cycle_2', 'Cycle_3', 'Cycle_4', 'Cycle_5']

sections = listdir(input_dir)
sections = [x for x in sections if x not in sections_to_remove]

In [5]:
sections=['689C_LNG3']

### specify where you want to put the files

In [6]:
output_dir = 'D:\Processing/'

# stack images 

## batch process

## zeiss

we need to make sure that we are stacking the correct files

# matlab processing: alignment, stitching and tiling

## add parameters

In [7]:
image_files = ('Base_1','Base_2','Base_3','Base_4','Base_5')

taglist = "C:/Users/chris.langseth/Downloads/Taglist_.csv"
nChannels = 5;
reference_round = 1;
tile_align_channel = 5; # for registration between rounds
tile_stitch_channel = 5; # Cy3   % for registration between tiles

## align tiles

In [8]:
output_dir

'D:\\Processing/'

In [9]:
align = align_2d_tiles.initialize()


In [10]:
for i, section in enumerate(sections):
    print(section)
    output_directory = output_dir + section + '/preprocessing/'
    metadatafile = pd.read_csv(output_directory +  '2DTiles' + '/tile_coordinates_Base_1.csv', header = None);
    nTiles = len(metadatafile)
    align.align_2d_tiles((output_directory + '2DTiles'),image_files, output_directory, nChannels, nTiles, reference_round, tile_align_channel, nargout = 0)


689C_LNG3


## stitch using the mist algorithm

In [11]:
stitch = stitch_2d_mist.initialize()

In [12]:
for i, section in enumerate(sections):
    print(section)
    output_directory = output_dir + section + '/preprocessing/'
    stitch.stitch_2d_mist((output_directory + '2DTiles'),image_files, output_directory, nChannels, tile_stitch_channel, reference_round, nargout = 0)

689C_LNG3


## tile stitched images

In [13]:
tile = tile_stitched_images.initialize()

In [14]:
for i, section in enumerate(sections):
    #try:
    print(section)
    output_directory = output_dir + section + '/preprocessing/'
    tile.tile_stitched_images((output_directory + 'Stitched2DTiles'),image_files, output_directory, float(6000), 'windows', nargout = 0)
    #except MatlabRunTimeError:
    #    continue
        

689C_LNG3


## run preliminary analysis

In [ ]:
decoding = [2,3,4,5]
from itertools import permutations  
perm = permutations(decoding) 
perm = list(perm)
perm

In [ ]:
perm = [(3, 4, 2, 5)]

In [ ]:
analysis = test_analysis.initialize()

In [ ]:
for i, section in enumerate(sections):
    print(section)
    for j, decoding_int in enumerate(perm):
        decoding_mod = (float(perm[j][0]), float(perm[j][1]), float(perm[j][2]), float(perm[j][3]))
        print(decoding_mod)
        output_directory = output_dir + section + '/preprocessing/'
        newMetaDataFile = pd.read_csv(output_directory +  'ReslicedTiles' + '/tilepos.csv', header = None)
        nTiles = len(newMetaDataFile)
        analysis.test_analysis((output_directory + 'ReslicedTiles'),image_files, output_directory, nTiles, (decoding_mod), taglist, 'windows', nargout = 0)


02A_Export
(3.0, 4.0, 2.0, 5.0)


## inspect data

In [ ]:
barcodes_df = pd.read_csv("C:/Users/chris.langseth/Documents/mouseDev/" + section + "/preprocessing/3425spots_with_coords_noNNNNs.csv")

In [ ]:
barcodes_df

,name,barcode,pos1,pos2
0,Fezf1,GCCTA,1861,540
1,Lmo2,TTGTT,1892,465
2,Fezf2,TGTTT,1901,456
3,Efna2,CCGCG,1915,526
4,Fezf2,TGTTT,1922,1524
...,...,...,...,...
58700,Ntf3,GCACT,7555,8514
58701,Tfap2c,GCAAG,7572,8473
58702,Fgf8,GTCCA,7572,9199
58703,Fgf8,GTCCA,7576,8792


In [ ]:
mpl.rcParams['figure.dpi'] = 500
grouped = barcodes_df.groupby('name')
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name1, group1 in grouped:
    ax.scatter(group1.pos1, group1.pos2, marker='.',linewidth = 0,s=0.05, label=name1)
#ax.legend()
plt.gca().invert_yaxis()
plt.axis('off')
plt.axis('scaled')
plt.show()